In [53]:
import pandas as pd
import numpy as np
import math
import prince
import time
from numpy.random import default_rng
import ipynb.fs.full.Functions_sara as fs
import ipynb.fs.full.data_preprocessing_after_imputing as dpai
import ipynb.fs.full.Adaboost as adaGB

# Adaptive Gradient Boosting

Using Adaptive Gradient Boosting to fit datasets. Firstly, fitting 8 imputed training data sets with validation set as parameter tuning data set to search hyper parameter grid. After tuning parameters, find the model with best performance and then employe the best parameter to fit a model using the combination of training and validation data set and test the model on our test data set. Thus the final model is based on the best model out of 8, with the best parameters.

In [76]:
# KNN
KNN_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/KNN_impute.csv")
KNN_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/KNN_impute_validation.csv")
KNN_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/KNN_impute_test.csv")


# Unconditional Mean
Mean_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/Unconditional_mean_impute.csv")
Mean_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/Unconditional_mean_impute_Validation.csv")
Mean_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/Unconditional_mean_impute_Test.csv")

# Listwise_deletion
Drop_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/Listwise_deletion.csv")
Drop_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/Listwise_deletion_Validation.csv")
Drop_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/Listwise_deletion_Test.csv")



In [8]:
def upsample(data):
    from sklearn.utils import resample
    
    df_majority=data[data.Default_ind==0]
    df_minority=data[data.Default_ind==1]
    largenumber=len(df_majority)
    df_minority_upsampled=resample(df_minority,replace=True,n_samples=int(1.2*largenumber),random_state=123)
    df_upsampled=pd.concat([pd.DataFrame(df_majority),pd.DataFrame(df_minority_upsampled)])
    return df_upsampled

In [77]:
KNN_train = upsample(KNN_train)
KNN_vali = upsample(KNN_vali)

Mean_train = upsample(Mean_train)
Mean_vali = upsample(Mean_vali)

Drop_train = upsample(Drop_train)
Drop_vali = upsample(Drop_vali)


In [78]:
KNN_train = dpai.Cate_to_object_excp_inq(KNN_train)
KNN_vali = dpai.Cate_to_object_excp_inq(KNN_vali)
KNN_test= dpai.Cate_to_object_excp_inq(KNN_test)
# Doesn't work
Mean_train = dpai.Cate_to_object_excp_inq(Mean_train)
Mean_vali = dpai.Cate_to_object_excp_inq(Mean_vali)
Mean_test= dpai.Cate_to_object_excp_inq(Mean_test)
# Doesn't work

Drop_train = dpai.Cate_to_object_excp_inq(Drop_train)
Drop_vali = dpai.Cate_to_object_excp_inq(Drop_vali)
Drop_test= dpai.Cate_to_object_excp_inq(Drop_test)

In [31]:
# Tunning Paramter Space
parameter_space = {
        'learning_rate':[0.1,0.01,0.001],
        'n_estimators': [50,100,150],     
        'algorithm':['SAMME.R','SAMME']
    }

In [32]:
def best_adaboost(datatrain, datavali, parameter_space,criteria='accuracy', randomstate=0):
    '''
    if not specified, the parameters are n_estimators=100, criterion="gini", max_depth=8, min_samples_split=400, 
    min_samples_leaf=50, bootstrap=True, oob_score=False, warm_start=False, max_samples=3000, random_state=0
    '''
    from sklearn.model_selection import PredefinedSplit
    import numpy as np
    from sklearn.ensemble import AdaBoostClassifier
    frames=pd.concat([datatrain, datavali])
    frames_reind=frames.set_index([pd.Index(range(len(frames)))])
    split_index=[-1]*len(datatrain)+[0]*len(datavali)
    pds = PredefinedSplit(test_fold = split_index)
    datadummy=pd.get_dummies(frames_reind)
    datatrainX=np.array(datadummy.drop('Default_ind', axis=1))
    datatrainy=datadummy['Default_ind']
    
    AGB=AdaBoostClassifier()
    from sklearn.model_selection import GridSearchCV
    #parameter_space = {
     #   'loss':['deviance', 'exponential'],
     #   'n_estimators': [100,300],
      #  'subsample': [0.5, 1],
       # #'criterion':['friedman_mse', 'mse', 'mae'], 
     #   'max_depth':range(1,10,2),
   # }
    modelAGS=GridSearchCV(AGB, parameter_space,cv=pds,scoring=criteria)
    modelAGS.fit(datatrainX, datatrainy)
    return modelAGS,datatrainX,datatrainy

# KNN

In [80]:
t = time.time()
adaKNN,x1,y1 = best_adaboost(KNN_train,KNN_vali,parameter_space)
elapsed = time.time() - t

In [81]:
print(elapsed)
adaKNN.score(x1,y1)

485.90358805656433


0.792282447823942

In [87]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(KNN_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaKNN.score(testX,testy))
y_pred=adaKNN.predict(testX)

confusion_matrix( testy,y_pred)

0.8028


array([[3683,  916],
       [  70,  331]])

In [ ]:
adaKNN.best_params_

In [121]:
# Combine Train+Validation data sets
frames=pd.concat([KNN_train, KNN_vali])
datadummy=pd.get_dummies(frames)
KNNX=np.array(datadummy.drop('Default_ind', axis=1))
KNNy=datadummy['Default_ind']
# best_param = {'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 100}

In [133]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaKNNf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=100)
adaKNNfull=adaKNNf.fit(KNNX,KNNy)
elapsed = time.time() - t

In [134]:
print(elapsed)
adaKNNfull.score(x1,y1)

6.999530792236328


0.7931004690199951

In [135]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(KNN_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaKNNfull.score(testX,testy))
y_pred=adaKNNfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8028


array([[3683,  916],
       [  70,  331]])

In [ ]:
y_pred.

In [192]:
916/(3683+ 916)

0.19917373342030875

# Mean

In [89]:
t = time.time()
adaMean,x1,y1 = best_adaboost(Mean_train,Mean_vali,parameter_space)
elapsed = time.time() - t

In [90]:
print(elapsed)
adaMean.score(x1,y1)

555.8331120014191


0.8019347504343536

In [91]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(Mean_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaMean.score(testX,testy))
y_pred=adaMean.predict(testX)

confusion_matrix( testy,y_pred)

0.7994


array([[3675,  924],
       [  79,  322]])

In [92]:
adaMean.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.2, 'n_estimators': 200}

In [136]:
# Combine Train+Validation data sets
frames=pd.concat([Mean_train, Mean_vali])
datadummy=pd.get_dummies(frames)
MeanX=np.array(datadummy.drop('Default_ind', axis=1))
Meany=datadummy['Default_ind']


In [140]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaMeanf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=200)
adaMeanfull=adaMeanf.fit(MeanX,Meany)
elapsed = time.time() - t

In [141]:
print(elapsed)
adaMeanfull.score(x1,y1)

13.530037879943848


0.7969883979264379

In [142]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(Mean_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaMeanfull.score(testX,testy))
y_pred=adaMeanfull.predict(testX)

confusion_matrix( testy,y_pred)

0.801


array([[3679,  920],
       [  75,  326]])

# ListWise Deletion

In [36]:
t = time.time()
adaDrop,x1,y1 = best_adaboost(Drop_train,Drop_vali,parameter_space)
elapsed = time.time() - t

0.802693121220246

In [49]:
adaDrop.score(x1,y1)

0.802693121220246

In [48]:
datadummy=pd.get_dummies(Drop_test)
testX=np.array(datadummy.drop('Default_ind', axis=1))
testy=datadummy['Default_ind']
adaDrop.score(testX,testy)

0.8084798843652132

In [45]:
y_pred=adaDrop.predict(testX)
from sklearn.metrics import confusion_matrix
confusion_matrix( testy,y_pred)

array([[3084,  726],
       [  69,  272]])

In [46]:
adaDrop.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150}

In [50]:
parameter_space = {
        'learning_rate':[0.2,0.15,0.1,0.07],
        'n_estimators': [100,150,200,250],     
        'algorithm':['SAMME.R','SAMME']
    }

In [54]:
t = time.time()
adaDrop2,x2,y2 = best_adaboost(Drop_train,Drop_vali,parameter_space)
elapsed = time.time() - t

In [56]:
print(elapsed)
adaDrop2.score(x2,y2)

502.656769990921


0.8074001251228885

In [57]:
# Confusion MAtrix
print(adaDrop2.score(testX,testy))
y_pred=adaDrop2.predict(testX)

confusion_matrix( testy,y_pred)

0.8094435075885329


array([[3090,  720],
       [  71,  270]])

In [58]:
adaDrop.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150}

In [143]:
# Combine Train+Validation data sets
frames=pd.concat([Drop_train, Drop_vali])
datadummy=pd.get_dummies(frames)
DropX=np.array(datadummy.drop('Default_ind', axis=1))
Dropy=datadummy['Default_ind']


In [144]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaDropf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=200)
adaDropfull=adaDropf.fit(DropX,Dropy)
elapsed = time.time() - t

In [145]:
print(elapsed)
adaDropfull.score(x1,y1)

28.660887956619263


0.7975808442359911

In [146]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(Drop_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaDropfull.score(testX,testy))
y_pred=adaDropfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8096844133943628


array([[3088,  722],
       [  68,  273]])

In [180]:
722/(3088+ 722)

0.18950131233595802

# Read in More Data

In [93]:
EM_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/EM_impute.csv")
EM_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/EM_validatiion.csv")
EM_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/EM_Impute.csv")

EMB_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/EMB_impute_Vali.csv")
EMB_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/EMB_impute_Test.csv")

RF_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/rf_Train.csv")
RF_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/rf_Vali.csv")
RF_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/rf_Test.csv")

Cart_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/ZCart_impute.csv")
Cart_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/ZCart_impute_Vali.csv")
Cart_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/ZCart_impute_Test.csv")

Median_train = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TrainData/Unconditional_median_impute.csv")
Median_vali = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/ValiData/Unconditional_median_impute_Validation.csv")
Median_test = pd.read_csv("/Users/BB10190915/Desktop/Data_Comp/TestData/Unconditional_median_impute_Test.csv")


In [94]:
EM_train = upsample(EM_train)
EM_vali = upsample(EM_vali)

EMB_train = upsample(EMB_train)
EMB_vali = upsample(EMB_vali)

RF_train = upsample(RF_train)
RF_vali = upsample(RF_vali)

Cart_train = upsample(Cart_train)
Cart_vali = upsample(Cart_vali)

Median_train = upsample(Median_train)
Median_vali = upsample(Median_vali)

In [95]:
# Factorization
EM_train = dpai.Cate_to_object_excp_inq(EM_train)
EM_vali = dpai.Cate_to_object_excp_inq(EM_vali)
EM_test= dpai.Cate_to_object_excp_inq(EM_test)
#####Input contains NaN, infinity or a value too large for dtype('float64').  NOOOOOOOO

EMB_train = dpai.Cate_to_object_excp_inq(EMB_train)
EMB_vali = dpai.Cate_to_object_excp_inq(EMB_vali)
EMB_test= dpai.Cate_to_object_excp_inq(EMB_test)

RF_train = dpai.Cate_to_object_excp_inq(RF_train)
RF_vali = dpai.Cate_to_object_excp_inq(RF_vali)
RF_test= dpai.Cate_to_object_excp_inq(RF_test)


Cart_train = dpai.Cate_to_object_excp_inq(Cart_train)
Cart_vali = dpai.Cate_to_object_excp_inq(Cart_vali)
Cart_test= dpai.Cate_to_object_excp_inq(Cart_test)


Median_train = dpai.Cate_to_object_excp_inq(Median_train)
Median_vali = dpai.Cate_to_object_excp_inq(Median_vali)
Median_test= dpai.Cate_to_object_excp_inq(Median_test)


In [83]:
# Tunning Paramter Space
parameter_space0 = {
        'learning_rate':[0.1,0.01,0.001],
        'n_estimators': [50,100,150],     
        'algorithm':['SAMME.R','SAMME']
    }

# EM

In [111]:
t = time.time()
adaEM,x1,y1 = best_adaboost(EM_train,EM_vali,parameter_space0)
elapsed = time.time() - t

In [112]:
print(elapsed)
adaEM.score(x1,y1)

175.529217004776


0.7979881510738089

In [113]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(EM_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaEM.score(testX,testy))
y_pred=adaEM.predict(testX)

confusion_matrix( testy,y_pred)

0.8026


array([[3686,  913],
       [  74,  327]])

In [115]:
adaEM.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150}

In [147]:
# Combine Train+Validation data sets
frames=pd.concat([EM_train, EM_vali])
datadummy=pd.get_dummies(frames)
EMX=np.array(datadummy.drop('Default_ind', axis=1))
EMy=datadummy['Default_ind']


In [160]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaEMf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=150)
adaEMfull=adaEMf.fit(EMX,EMy)
elapsed = time.time() - t

In [161]:
print(elapsed)
adaEMfull.score(x1,y1)

22.03677535057068


0.7979881510738089

In [162]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(EM_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaEMfull.score(testX,testy))
y_pred=adaEMfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8026


array([[3686,  913],
       [  74,  327]])

# EMB

In [68]:
t = time.time()
adaEMB,x1,y1 = best_adaboost(EMB_train,EMB_vali,parameter_space0)
elapsed = time.time() - t

In [69]:
print(elapsed)
adaEMB.score(x1,y1)

178.66418313980103


0.7916604105446043

In [70]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(EMB_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaEMB.score(testX,testy))
y_pred=adaEMB.predict(testX)

confusion_matrix( testy,y_pred)

0.8042


array([[3693,  906],
       [  73,  328]])

In [71]:
adaEMB.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150}

In [151]:
# Combine Train+Validation data sets
frames=pd.concat([EMB_train, EMB_vali])
datadummy=pd.get_dummies(frames)
EMBX=np.array(datadummy.drop('Default_ind', axis=1))
EMBy=datadummy['Default_ind']


In [157]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaEMBf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=150)
adaEMBfull=adaEMBf.fit(EMBX,EMBy)
elapsed = time.time() - t

In [158]:
print(elapsed)
adaEMBfull.score(x1,y1)

12.989346981048584


0.7944705011108368

In [159]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(EMB_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaEMBfull.score(testX,testy))
y_pred=adaEMBfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8014


array([[3680,  919],
       [  74,  327]])

In [196]:
919/(3680+  919)

0.19982604914111762

# RF dataset

In [72]:
t = time.time()
adaRF,x1,y1 = best_adaboost(RF_train,RF_vali,parameter_space0)
elapsed = time.time() - t

In [73]:
print(elapsed)
adaRF.score(x1,y1)


159.11242508888245


0.7958001758864032

In [74]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(RF_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaRF.score(testX,testy))
y_pred=adaRF.predict(testX)

confusion_matrix( testy,y_pred)

0.8008


array([[3679,  920],
       [  76,  325]])

In [75]:
adaRF.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 150}

In [156]:
# Combine Train+Validation data sets
frames=pd.concat([RF_train, RF_vali])
datadummy=pd.get_dummies(frames)
RFX=np.array(datadummy.drop('Default_ind', axis=1))
RFy=datadummy['Default_ind']


In [166]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaRFf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=150)
adaRFfull=adaRFf.fit(RFX,RFy)
elapsed = time.time() - t

In [167]:
print(elapsed)
adaRFfull.score(x1,y1)

12.790457010269165


0.795988644779067

In [168]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(RF_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaRFfull.score(testX,testy))
y_pred=adaRFfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8008


array([[3679,  920],
       [  76,  325]])

# Cart

In [97]:
t = time.time()
adaCart,x1,y1 = best_adaboost(Cart_train,Cart_vali,parameter_space0)
elapsed = time.time() - t

In [98]:
print(elapsed)
adaCart.score(x1,y1)


177.0503761768341


0.7923038973852985

In [99]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(Cart_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaCart.score(testX,testy))
y_pred=adaCart.predict(testX)

confusion_matrix( testy,y_pred)

0.8036


array([[3689,  910],
       [  72,  329]])

In [100]:
adaCart.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 100}

In [169]:
# Combine Train+Validation data sets
frames=pd.concat([Cart_train, Cart_vali])
datadummy=pd.get_dummies(frames)
CartX=np.array(datadummy.drop('Default_ind', axis=1))
Carty=datadummy['Default_ind']


In [170]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaCartf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=150)
adaCartfull=adaCartf.fit(CartX,Carty)
elapsed = time.time() - t

In [171]:
print(elapsed)
adaCartfull.score(x1,y1)

10.269256114959717


0.7969513700320908

In [172]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(Cart_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaCartfull.score(testX,testy))
y_pred=adaCartfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8006


array([[3676,  923],
       [  74,  327]])

In [190]:
923/(3676+ 923)

0.20069580343552945

# Median

In [101]:
t = time.time()
adaMedian,x1,y1 = best_adaboost(Median_train,Median_vali,parameter_space0)
elapsed = time.time() - t

In [102]:
print(elapsed)
adaMedian.score(x1,y1)

160.8201470375061


0.7928401364192103

In [103]:
# Prediction and Confusion MAtrix
datadummy = pd.get_dummies(Median_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaMedian.score(testX,testy))
y_pred=adaMedian.predict(testX)

confusion_matrix( testy,y_pred)

0.8044


array([[3693,  906],
       [  72,  329]])

In [104]:
adaMedian.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 100}

In [173]:
# Combine Train+Validation data sets
frames=pd.concat([Median_train, Median_vali])
datadummy=pd.get_dummies(frames)
MedianX=np.array(datadummy.drop('Default_ind', axis=1))
Mediany=datadummy['Default_ind']

In [174]:
t=time.time()
from sklearn.ensemble import AdaBoostClassifier
adaMedianf = AdaBoostClassifier(algorithm='SAMME.R', learning_rate=0.1,n_estimators=150)
adaMedianfull=adaMedianf.fit(MedianX,Mediany)
elapsed = time.time() - t

In [175]:
print(elapsed)
adaMedianfull.score(x1,y1)

10.535698890686035


0.7980375215996051

In [176]:
# Prediction and Confusion Matrix
datadummy = pd.get_dummies(Median_test)
testX = np.array(datadummy.drop('Default_ind', axis=1))
testy = datadummy['Default_ind']

print(adaMedianfull.score(testX,testy))
y_pred=adaMedianfull.predict(testX)

confusion_matrix( testy,y_pred)

0.8008


array([[3677,  922],
       [  74,  327]])

In [183]:
922/(3677+  922)

0.2004783648619265

# Tune more parameters

In [106]:
parameter_space2 = {
        'learning_rate':[0.15,0.125,0.1,0.075,0.05],
        'n_estimators': [80,90,100,120,130],     
        'algorithm':['SAMME.R']
    }

In [107]:
t = time.time()
adaKNN2,x2,y2 = best_adaboost(KNN_train,KNN_vali,parameter_space2)
elapsed = time.time() - t

In [108]:
print(elapsed)
adaKNN2.score(x2,y2)

278.73006105422974


0.792282447823942

In [109]:
# Prediction and Confusion MAtrix

print(adaKNN2.score(testX,testy))
y_pred=adaKNN2.predict(testX)

confusion_matrix( testy,y_pred)

0.8036


array([[3687,  912],
       [  70,  331]])

In [110]:
adaKNN2.best_params_

{'algorithm': 'SAMME.R', 'learning_rate': 0.1, 'n_estimators': 100}